# Source: Rivals

In [ ]:
#hide
import core_constants as cc
import functions as fx
import json

## Set Notebook Settings

In [ ]:
conference = 'pactwelve'

years = cc.get_defYears()
headers= cc.get_header()
schoolsList = cc.get_schoolsList()
teamDirectory = cc.get_htmlDir('rivals', conference, 'teams')
playerDirectory = cc.get_htmlDir('rivals', conference, 'recruits')

## Get & Save the Teams & Players Page HTML
#### Source: https://maryland.rivals.com/commitments/football/2012
> This page contains metadata of each player along with the Rivals ranking and stars.  Unlike 247Sports, we process the fetch and save of both pages directly from a single function

In [ ]:
fx.get_Rivals(conference, schoolsList, years, headers, sleepyTime=6)

## Process Local Rivals HTML Files

> All of this processing is done locally, using the files saved in the previous few steps.  This creates an exhaustive store of all the fields grabbed from the scrapes.

In [ ]:
cc.save_records('scrapedData', 'rivals_' + conference, fx.process_Rivals(playerDirectory, conference, schoolsList))

## Summarize the Dataset

> We don't need repetitive fields across the various datasets (ie - I don't need height coming back from 3 sources).  This step strips to only what I care about for the master print out.

In [ ]:
outputDir = '..//summarizedData//'
dataset = 'rivals'

print(fx.summarize_Rivals())

#with open(outputDir + dataset + "data.json", "w", encoding="utf-8") as write_file:
#                write_file.write(json.dumps(fx.summarize_Rivals()))

{'school': None, 'year': '2013', 'playerName': 'Alex Gall', 'city': 'Cincinnati', 'state': 'OH', 'highSchool': 'Moeller', 'position': 'OL', 'height': 77.0, 'weight': 295, 'stars': 3, 'nationalRank': None, 'positionRank': None, 'stateRank': None}
{'school': None, 'year': '2008', 'playerName': 'Marcus Robinson', 'city': 'Homestead', 'state': 'FL', 'highSchool': 'Homestead', 'position': 'LB', 'height': 74.0, 'weight': 210, 'stars': 4, 'nationalRank': 65, 'positionRank': 4, 'stateRank': 11}
{'school': None, 'year': '2002', 'playerName': 'Sinorice Moss', 'city': 'Miami', 'state': 'FL', 'highSchool': 'Carol City', 'position': 'WR', 'height': 68.0, 'weight': 163, 'stars': 3, 'nationalRank': None, 'positionRank': 71, 'stateRank': None}
{'school': None, 'year': '2017', 'playerName': 'Deejay Dallas', 'city': 'Brunswick', 'state': 'GA', 'highSchool': 'Glynn Academy', 'position': 'ATH', 'height': 72.0, 'weight': 182, 'stars': 4, 'nationalRank': 196, 'positionRank': 16, 'stateRank': 19}
{'school': 

TypeError: can only concatenate str (not "NoneType") to str